In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/Users/weilu/openmmawsem/')

from openmmawsem import *

In [3]:
class ForceReporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, True, False)

    def report(self, simulation, state):
        forces = state.getForces().value_in_unit(kilocalories/mole/nanometer)
        self._out.write(str(state.getTime())+"\n")
        #self._out.write(str(dir(state)))
        for f in forces:
            self._out.write('%g %g %g\n' % (f[0], f[1], f[2]))

In [4]:
# build OpenMMAWSEM simulation object
# check each energy term individually
pdb_id = '2xov'
chain_id = 'A'
# prepare pdb
# res_names = prepare_pdb("%s.pdb" % pdb_id, [chain_id])

def openawsemsystem(pdb='2xov.pdb',chain='A',term=-1):
    resnames=prepare_pdb(pdb,chain)
    output_pdb_filename = pdb.split('.')[0] + '-openmmawsem.pdb'
    oa = OpenMMAWSEMSystem(output_pdb_filename,resnames, xml_filename="../../awsem.xml")

    # setup Hamiltonian
    if term==1:
        print('apply_con_term')
        term_name='Con'
        apply_con_term(oa)
    elif term==2:
        print('apply_chain_term')
        term_name='Chain'
        apply_chain_term(oa)
    elif term==3:
        print('apply_chi_term')
        term_name='Chi'
        apply_chi_term(oa)
    elif term==4:
        print('apply_excl_term')
        term_name='Excluded'
        apply_excl_term(oa)
    elif term==5:
        print('apply_rama_term')
        term_name='Rama'
        apply_rama_term(oa)
    elif term==6:
        print('apply_direct_term')
        term_name='Direct'
        apply_direct_term(oa)
    elif term==7:
        print('apply_mediated_term')
        term_name='Mediated'
        apply_mediated_term(oa)
    elif term==8:
        print('apply_contact_term')
        term_name='Contact'
        apply_contact_term(oa)
    elif term==9:
        print('apply_beta_term')
        term_name='Beta'
        apply_beta_term(oa)
    elif term==10:
        print('apply_pap_term')
        term_name='P_AP'
        apply_pap_term(oa)
    elif term==11:
        print('apply_dsb_term')
        term_name='DSB'
        apply_dsb_term(oa)
    elif term==12:
        print('apply_helix_term')
        term_name='HELIX'
        apply_helix_term(oa)
    elif term==13:
        print('apply_associative_memory_term')
        term_name='Frag_Mem'
        apply_associative_memory_term(oa)
    return term_name,oa
import time

OA_energy={}

for i in range(5,6):
    if i==12:
        continue
        #pass
    term_name,oa=openawsemsystem(term=i)
    # setup simulation parameters
    reporter_frequency = 1000
    num_steps = 1000
    step_size = 2e-10*femtoseconds
    temperature = 100*kelvin
#     platform = Platform.getPlatformByName('Reference') # OpenCL, CUDA, CPU, or Reference

    # start simulation
    #integrator = VerletIntegrator(step_size)
    integrator = LangevinIntegrator(temperature, 1/picosecond, step_size)
    #simulation = Simulation(oa.pdb.topology, oa.system, integrator, platform)
    simulation = Simulation(oa.pdb.topology, oa.system, integrator)
    simulation.context.setPositions(oa.pdb.positions)
    #simulation.context.setVelocitiesToTemperature(temperature, 1)
    #simulation.minimizeEnergy(tolerance=Quantity(value=.0001, unit=kilojoule/mole))
    simulation.reporters.append(PDBReporter('output.pdb', reporter_frequency))
    #simulation.reporters.append(ForceReporter('forces.txt', 100))
    simulation.reporters.append(StateDataReporter(stdout, reporter_frequency, step=True,
            potentialEnergy=True, temperature=True))
    tt=time.time()
    simulation.step(1)
    state = simulation.context.getState(getEnergy=True)
#     print((time.time()-tt)*1000,'ms')
#     print(term_name, state.getPotentialEnergy())
    print(term_name, state.getPotentialEnergy().value_in_unit(kilocalories/mole), "kcal/mol")
    OA_energy[term_name]=state.getPotentialEnergy()._value
    #print (i,time.time()-tt)

apply_rama_term
Rama -297.87668260971856 kcal/mol


In [ ]:
297.876/302.69989

In [6]:
pd.read_csv("energy.dat").head()

,Steps,Chain,Chi,Rama,Water,Helix,AMH-Go,Frag_Mem,Membrane,VTotal,Ebond,Epair,Rg
0,0,104.187332,22.560808,-302.699897,0,0,-579.196818,0,-33.468043,-788.616619,204.796601,241.126954,12.613729
1,24000,129.388783,35.275016,-345.147553,0,0,-466.461385,0,-40.556127,-687.501267,209.667439,14.555298,9.944762
2,48000,146.283416,35.346898,-345.182024,0,0,-487.521341,0,-37.488319,-688.561369,206.245782,16.341251,10.479956
3,72000,121.421910,37.691164,-351.544374,0,0,-482.385635,0,-36.047035,-710.863969,208.335979,21.326854,12.757063
4,96000,118.788308,31.115704,-342.291178,0,0,-481.868033,0,-38.208300,-712.463499,195.815988,13.142091,10.248300


In [7]:
data = pd.read_table("2xov-openmmawsem.pdb", sep="\s+", header=None, names=["_","i","type","res","chain","res_id", "x","y","z","_","_","_"])

In [9]:
data = data.dropna().reset_index()
data["res_id"] = data["res_id"].apply(lambda x: int(x))

In [17]:
chosen = data.query("res_id < 3")
a = chosen[["x","y","z"]].values
b = a[[1,4,6,7]]
chosen

,index,_,i,type,res,chain,res_id,x,y,z,_.1,_.2,_.3
0,0,ATOM,1.0,CA,NGP,A,1,12.490,-0.323,-20.024,1.0,0.0,C
1,1,ATOM,2.0,C,NGP,A,1,12.800,-0.437,-18.529,1.0,0.0,C
2,2,ATOM,3.0,O,NGP,A,1,13.012,-1.499,-17.996,1.0,0.0,O
3,3,ATOM,4.0,CB,NGP,A,1,11.065,-0.802,-20.300,1.0,0.0,B
4,4,ATOM,5.0,N,NGP,A,2,12.819,0.686,-17.881,1.0,0.0,N
5,5,ATOM,6.0,H,NGP,A,2,12.648,1.543,-18.312,1.0,0.0,H
6,6,ATOM,7.0,CA,NGP,A,2,13.095,0.800,-16.438,1.0,0.0,C
7,7,ATOM,8.0,C,NGP,A,2,11.996,1.573,-15.719,1.0,0.0,C
8,8,ATOM,9.0,O,NGP,A,2,11.222,2.281,-16.323,1.0,0.0,O
9,9,ATOM,10.0,CB,NGP,A,2,14.423,1.509,-16.197,1.0,0.0,B


In [37]:
def compute_dihedral(p):
    """Praxeolitic formula
    1 sqrt, 1 cross product"""
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.arctan2(y, x)
compute_dihedral(b)

-2.209245819114686

In [11]:
def compute_chi(data):
    ca_all = data.query("type == 'CA'")[["x","y","z"]].values
    cb_all = data.query("type == 'CB'")[["x","y","z"]].values
    c_all = data.query("type == 'C'")[["x","y","z"]].values
    n_all = data.query("type == 'N'")[["x","y","z"]].values
    energy = 0 
    for i in range(len(n_all)):
        ca = ca_all[i]
        cb = cb_all[i]
        c = c_all[i]
        n = n_all[i]
        chi0 = -0.71
        k_chi = 60*4.184
        r_ca_cb = cb-ca
        r_c_ca = ca-c
        r_ca_n = n-ca
        norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
        norm_r_c_ca = np.sum(r_c_ca**2)**0.5
        norm_r_ca_n = np.sum(r_ca_n**2)**0.5
        a = np.cross(-r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
        chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
        dchi = chi - chi0
        energy += k_chi*dchi*dchi
    return energy

In [12]:
chosen = data.query("res != 'IGL' and res_id != 1 and res_id != 181")
compute_chi(chosen)/4.184

22.490468654328367